In [1]:
!git clone https://github.com/xpww/ECG_BNN_Net.git

Cloning into 'ECG_BNN_Net'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 20 (delta 3), reused 17 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (20/20), 21.59 MiB | 51.18 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [2]:
!tar -xvf /content/ECG_BNN_Net/ECG_Dataset.tar.xz

Streaming output truncated to the last 5000 lines.
ECG_Dataset/ECG-5/1 N/N_4707.npy
ECG_Dataset/ECG-5/1 N/N_4784.npy
ECG_Dataset/ECG-5/1 N/N_4819.npy
ECG_Dataset/ECG-5/1 N/N_4885.npy
ECG_Dataset/ECG-5/1 N/N_4902.npy
ECG_Dataset/ECG-5/1 N/N_4917.npy
ECG_Dataset/ECG-5/1 N/N_4967.npy
ECG_Dataset/ECG-5/1 N/N_4995.npy
ECG_Dataset/ECG-5/1 N/N_4996.npy
ECG_Dataset/ECG-5/1 N/N_503.npy
ECG_Dataset/ECG-5/1 N/N_5034.npy
ECG_Dataset/ECG-5/1 N/N_5037.npy
ECG_Dataset/ECG-5/1 N/N_5055.npy
ECG_Dataset/ECG-5/1 N/N_5105.npy
ECG_Dataset/ECG-5/1 N/N_563.npy
ECG_Dataset/ECG-5/1 N/N_583.npy
ECG_Dataset/ECG-5/1 N/N_638.npy
ECG_Dataset/ECG-5/1 N/N_731.npy
ECG_Dataset/ECG-5/1 N/N_737.npy
ECG_Dataset/ECG-5/1 N/N_751.npy
ECG_Dataset/ECG-5/1 N/N_802.npy
ECG_Dataset/ECG-5/1 N/N_83.npy
ECG_Dataset/ECG-5/1 N/N_89.npy
ECG_Dataset/ECG-5/1 N/N_931.npy
ECG_Dataset/ECG-5/1 N/N_94.npy
ECG_Dataset/ECG-5/1 N/N_955.npy
ECG_Dataset/ECG-5/1 N/N_963.npy
ECG_Dataset/ECG-5/1 N/N_979.npy
ECG_Dataset/ECG-5/1 N/N_1012.npy
ECG_Datase

In [3]:
import torch
import copy
import itertools
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt

from torch.optim import Adam
from mpl_toolkits import mplot3d
from prettytable import PrettyTable
from scipy.interpolate import make_interp_spline

import numpy
# import loss_landscapes
# import loss_landscapes.metrics
import pandas as pd
def plot_confusion_matrix(cm, classes, cmap,  index,normalize=False, title='Confusion matrix'):
	cm_nor = cm.astype('float') / cm.sum(axis=0)
	# cm_nor = cm.astype('float')
	sum_TP = 0
	for i in range(len(classes)):
		sum_TP += cm[i, i]
	acc = sum_TP / np.sum(cm)
	acc_str = "{:.2f}".format(acc * 100) + '%'
	print("the model accuracy is ", acc)
	table = PrettyTable()
	table.field_names = ["", "ACC", "SEN", "SPE", "PPR", "F1"]
	TP_SUM, FP_SUM, FN_SUM, TN_SUM = 0, 0, 0, 0
	ACC_Tol, SEN_Tol, SPE_Tol, PPR_Tol, F1_Tol = 0, 0, 0, 0, 0
	for i in range(len(classes)):
		TP = cm[i, i]
		FP = np.sum(cm[i, :]) - TP
		FN = np.sum(cm[:, i]) - TP
		TN = np.sum(cm) - TP - FP - FN
		TP_SUM += TP
		FP_SUM += FP
		FN_SUM += FN
		TN_SUM += TN

		ACC = round((TP + TN) / (TP + FP + TN + FN), 3) if TP + FP + TN + FN != 0 else 0.
		SEN = round(TP / (TP + FN), 3) if TP + FN != 0 else 0.
		SPE = round(TN / (TN + FP), 3) if TN + FP != 0 else 0.
		PPR = round(TP / (TP + FP), 3) if TP + FP != 0 else 0.
		F1 = round(2 * TP / (2 * TP + FP + FN), 3) if TP + FP != 0 else 0.
		ACC_Tol += ACC
		SEN_Tol += SEN
		SPE_Tol += SPE
		PPR_Tol += PPR
		F1_Tol += F1
		table.add_row([classes[i], ACC, SEN, SPE, PPR, F1])
	ACC_SUM = round((TP_SUM + TN_SUM) / (TP_SUM + FP_SUM + TN_SUM + FN_SUM),
					3) if TP_SUM + FP_SUM + TN_SUM + FN_SUM != 0 else 0.
	SEN_SUM = round(TP_SUM / (TP_SUM + FN_SUM), 3) if TP_SUM + FN_SUM != 0 else 0.
	SPE_SUM = round(TN_SUM / (TN_SUM + FP_SUM), 3) if TN_SUM + FP_SUM != 0 else 0.
	PPR_SUM = round(TP_SUM / (TP_SUM + FP_SUM), 3) if TP_SUM + FP_SUM != 0 else 0.
	F1_SUM = round(2 * TP_SUM / (2 * TP_SUM + FP_SUM + FN_SUM), 3) if TP_SUM + FP_SUM != 0 else 0.
	ACC_MEAN = round(ACC_Tol/len(classes), 3)
	SEN_MEAN = round(SEN_Tol/len(classes), 3)
	SPE_MEAN = round(SPE_Tol/len(classes), 3)
	PPR_MEAN = round(PPR_Tol/len(classes), 3)
	F1_MEAN = round(F1_Tol/len(classes), 3)
	table.add_row(['SUM', ACC_SUM, SEN_SUM, SPE_SUM, PPR_SUM, F1_SUM])
	table.add_row(['MEAN', ACC_MEAN, SEN_MEAN, SPE_MEAN, PPR_MEAN, F1_MEAN])
	print(table)
	plt.imshow(cm_nor, interpolation='nearest', cmap=cmap)
	plt.title(title)
	plt.colorbar(fraction=0.046, pad=0.05)
	tick_marks = np.arange(len(classes))
	plt.xticks(tick_marks, classes, rotation=45)
	plt.yticks(tick_marks, classes)
	thresh = cm_nor.max() / 2.
	length = len(classes)
	if length == 5:
		fontsize = 13
	elif length == 17:
		fontsize = 6
	for i, j in itertools.product(range(cm_nor.shape[0]), range(cm_nor.shape[1])):
		plt.text(j, i, format(cm_nor[i, j], '.2f'), horizontalalignment="center",
				 color="white" if cm_nor[i, j] > thresh else "black", fontsize=fontsize)
	plt.tight_layout()
	plt.ylabel('Predicted Labels', fontsize=fontsize)
	plt.xlabel('True Labels', fontsize=fontsize)
	plt.title('Normalized confusion matrix, with Acc=%.2f' % (100*acc), fontsize=13)

def confusion_matrix(preds, labels, conf_matrix):
	for p, t in zip(preds, labels):
		conf_matrix[p, t] += 1

	return conf_matrix


def plot_cfm(model, test_loader, classes, mode, cnt, clt, seed, best_test_acc, index):
	length = len(classes)
	conf_matrix = torch.zeros(length, length)
	acc_val = 0
	model.eval()
	total = 0
	correct = 0
	for data in test_loader:
		images, labels = data
		total += len(labels)
		out = model(images)
		prediction = torch.max(out, 1)[1]
		conf_matrix = confusion_matrix(
			prediction, labels=labels, conf_matrix=conf_matrix)
		correct += (prediction == labels).sum().item()
	acc_val = 100 * correct / total
	print(acc_val)
	attack_types = classes
	plt.tight_layout()
	plt.figure(figsize=(6, 6))
	plt.rcParams['font.sans-serif'] = ['Times New Roman']
	plot_confusion_matrix(
		conf_matrix.numpy(), classes=attack_types, cmap=plt.cm.Blues, normalize=True, index = index)

	plt.savefig(fname=f"./image_for_{len(classes)}/%s,Confusion-matrix,%d,%d,%d,%.4f.pdf" % (mode, cnt, clt, seed, best_test_acc), format="pdf", bbox_inches='tight')


def plot_loss_acc(acc_list, loss_list, mode, cnt, clt):
	fig = plt.figure()
	x = np.arange(1, len(acc_list) + 1)
	a1 = fig.add_axes([0, 0, 1, 1])
	a1.plot(x, acc_list, 'tab:blue', label='acc')
	a1.set_ylabel('acc')
	a2 = a1.twinx()
	a2.plot(x, loss_list, 'tab:orange', label='loss')
	a2.set_ylabel('loss')
	plt.title('acc & loss')
	a1.set_xlabel('Epoch')
	a1.legend()
	a2.legend()
	plt.savefig(fname="../image/%s,Acc-Loss,%d,%d.pdf" % (mode, cnt, clt), format="pdf", bbox_inches='tight')
	plt.show()


In [4]:
import math
from typing import Union
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Function
from torch.nn.common_types import _size_1_t


# -------------------------------------STE-----------------------------------
class BinarizeF(Function):

    @staticmethod
    def forward(ctx, input):
        # output = input.new(input.size())
        # output[input >= 0] = 1
        # output[input < 0] = -1
        ctx.save_for_backward(input)

        return input.sign()

    @staticmethod
    def backward(ctx, grad_output):
        # grad_input = grad_output.clone()
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input.masked_fill_(input.ge(1) | input.le(-1), 0)
        return grad_input


# ---------------------------------(2/coshx) ^2--------------------------
class bcEstimator(Function):  # approx with (2/coshx) ^2 like in binaryConnect?
    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        out = torch.sign(input)
        return out

    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        grad_input = (2 / torch.cosh(input)) ** 2
        return grad_output * grad_input


# ---------------------------------tanhx--------------------------
class irEstimator(Function):  # approx with (2/coshx) ^2 like in binaryConnect?
    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        out = torch.sign(input)
        return out

    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        grad_input = 1 - torch.pow(torch.tanh(input), 2)
        return grad_output * grad_input


# ---------------------------------fft--------------------------
class fft_Estimator(Function):
    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        out = torch.sign(input)
        return out

    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        # grad_input = 0
        # for i in range(0, 10):
        #     grad_input += (4/torch.pi) * torch.cos(input=(2*i+1)*input)
        pi = torch.pi
        # grad_input = (4 / pi) * (torch.cos(input=(2 * 0 + 1) * pi * input) + torch.cos(input=(2 * 1 + 1) * pi * input)
        #                                +torch.cos(input=(2 * 2 + 1) * input) + torch.cos(input=(2 * 3 + 1) * input)
        #                                + torch.cos(input=(2 * 4 + 1) * input) + torch.cos(input=(2 * 5 + 1) * input)
        #                                + torch.cos(input=(2 * 6 + 1) * input) + torch.cos(input=(2 * 7 + 1) * input)
        #                                )
        # grad_input = (4 / pi) * (torch.cos(input=(2 * 0 + 1) * input) + torch.cos(input=(2 * 1 + 1) * input)
        #                          + torch.cos(input=(2 * 2 + 1) * input)
        #                          )
        grad_input = (pi / 2) * torch.cos((pi / 2) * grad_output)
        grad_input.masked_fill_(input.ge(1) | input.le(-1), 0)
        return grad_output * grad_input


# ---------------------------------2x+-x^2--------------------------
class BinaryActivation(nn.Module):
    def __init__(self):
        super(BinaryActivation, self).__init__()

    def forward(self, x):
        out_forward = torch.sign(x)
        # out_e1 = (x^2 + 2*x)
        # out_e2 = (-x^2 + 2*x)
        # out_e_total = 0
        mask1 = x < -1
        mask2 = x < 0
        mask3 = x < 1
        out1 = (-1) * mask1.type(torch.float32) + (x * x + 2 * x) * (1 - mask1.type(torch.float32))
        out2 = out1 * mask2.type(torch.float32) + (-x * x + 2 * x) * (1 - mask2.type(torch.float32))
        out3 = out2 * mask3.type(torch.float32) + 1 * (1 - mask3.type(torch.float32))
        out = out_forward.detach() - out3.detach() + out3

        return out


class IR_Estimator(Function):  # implementing BiRealNet paper approximation
    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        out = torch.sign(input)
        return out

    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        mask1 = (input >= -1) * 1
        mask2 = (input >= 0) * 1
        mask3 = (input >= 1) * 1

        grad_input = (mask1 - mask2) * (2 + 2 * input) + (mask2 - mask3) * (2 - 2 * input)
        # grad_input.masked_fill_(input.ge(1) | input.le(-1), 0)
        return grad_output * grad_input


binarize = BinarizeF.apply

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


def bin_act(x):
    bin_act = torch.sign(x).detach()
    le_clip = x.lt(-1.0).type(torch.float32)
    ri_clip = x.ge(1.0).type(torch.float32)
    clip_l = torch.bitwise_and(x.ge(-1.0), x.lt(0.0))
    clip_r = torch.bitwise_and(x.ge(0.0), x.lt(1.0))
    cliped = clip_l * (2 + x) * x + clip_r * (2 - x) * x
    out = cliped + ri_clip - le_clip
    # out = torch.tanh(x)
    return bin_act + out - out.detach()


def bin_wei(x):
    bin_wei = torch.sign(x).detach()
    out = torch.tanh(x)
    return bin_wei + out - out.detach()


class BinActivation(nn.Module):
    def __init__(self):
        super(BinActivation, self).__init__()

    def forward(self, x):
        out = bin_wei(x)
        return out
class FocalLoss(nn.Module):

    def __init__(self, weight=None, reduction='mean', gamma=0, alpha=1, eps=1e-7):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.eps = eps
        self.ce = torch.nn.CrossEntropyLoss(weight=weight, reduction=reduction)

    def forward(self, input, target):
        logp = self.ce(input, target)
        p = torch.exp(-logp)
        loss = self.alpha * (1 - p) ** self.gamma * logp
        return loss.mean()

class focal_loss(nn.Module):
    def __init__(self, alpha, gamma):
        super(focal_loss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.BCE = nn.BCEWithLogitsLoss(reduction='sum')

    def forward(self, logits, labels):
        BCloss = self.BCE(input=logits.float(), target=labels.float())

        if self.gamma == 0:
            modulator = 1.0
        else:
            modulator = torch.exp(-self.gamma * labels * logits - self.gamma * torch.log(1 + torch.exp(-1.0 * logits)))

        loss = modulator * BCloss

        weighted_loss = self.alpha * loss
        fl = torch.sum(weighted_loss)

        fl /= torch.sum(labels)

        return

import numpy as np
from scipy.stats import ortho_group
def get_ab(N):
    sqrt = int(np.sqrt(N))
    for i in range(sqrt, 0, -1):
        if N % i == 0:
            return i, N // i
class BinaryConv1d_RBNN(nn.Conv1d):
    def __init__(self, in_channels: int, out_channels: int, kernel_size: _size_1_t, stride: _size_1_t = 1,
                 padding: Union[str, _size_1_t] = 0,
                 dilation: _size_1_t = 1,
                 groups: int = 1,
                 bias: bool = False,
                 padding_mode: str = 'zeros', ):
        super().__init__(in_channels, out_channels, kernel_size, stride=stride, padding=padding, dilation=dilation,
                         groups=groups, bias=bias, padding_mode=padding_mode)
        # RBNN
        self.k = torch.tensor([10.]).float()
        self.t = torch.tensor([0.1]).float()
        self.epoch = -1

        w = self.weight
        self.a, self.b = get_ab(np.prod(w.shape[1:]))
        if self.a == 1:
            R1 = torch.tensor([[1.0]]).float().cuda()
        else:
            R1 = torch.tensor(ortho_group.rvs(dim=self.a)).float().cuda()
        R2 = torch.tensor(ortho_group.rvs(dim=self.b)).float().cuda()
        self.register_buffer('R1', R1)

        self.register_buffer('R2', R2)
        self.Rweight = torch.ones_like(w)

        self.w3 = torch.ones_like(w).cuda()

        self.rotate = nn.Parameter(torch.ones(w.size(0), 1, 1).cuda() * np.pi / 2, requires_grad=True)
        self.Rotate = torch.zeros(1)
    def forward(self, input):
        # a = input
        # w = self.weight
        # -------IR-Net--------
        # ba = IR_Estimator().apply(a)
        # bw = IR_Estimator().apply(w)
        # -------STE----------
        # if self.training:
        if self.in_channels == 1:
            ba = input
        else:
            ba = BinarizeF().apply(input)

        # a0 = input
        if self.training:
            w = self.weight
            a, b = self.a, self.b
            X = w.view(w.shape[0], a, b).detach()
            if self.epoch > -1:
                for _ in range(3):
                    # * update B
                    V = self.R1.t() @ X.detach() @ self.R2
                    B = torch.sign(V)
                    # * update R1
                    D1 = sum([Bi @ (self.R2.t()) @ (Xi.t()) for (Bi, Xi) in zip(B, X.detach())])
                    U1, S1, V1h = torch.linalg.svd(D1)
                    V1 = V1h.mH
                    self.R1 = (V1 @ (U1.t()))
                    # * update R2
                    D2 = sum([(Xi.t()) @ self.R1 @ Bi for (Xi, Bi) in zip(X.detach(), B)])
                    U2, S2, V2h = torch.linalg.svd(D2)
                    V2 = V2h.mH
                    self.R2 = (U2 @ (V2.t()))
            self.Rweight = ((self.R1.t()) @ X @ (self.R2)).view_as(w)
            delta = self.Rweight.detach() - w
            self.w3 = w + torch.abs(torch.sin(self.rotate)) * delta
            bw = BinarizeF().apply(self.w3)
        else:
            bw = BinarizeF().apply(self.w3)
        # else:
        #     ba = torch.sign(a)
        #     bw = torch.sign(w)
        # -------2/coshx----------
        # ba = bcEstimator().apply(a)
        # bw = bcEstimator().apply(w)
        # -----------tanhx-----------
        # ba = irEstimator().apply(a)
        # bw = irEstimator().apply(w)
        # -----------fft-----------
        # ba = fft_Estimator().apply(a)
        # bw = fft_Estimator().apply(w)

        out = F.conv1d(input=ba, weight=bw, bias=None, stride=self.stride,
                       padding=self.padding, dilation=self.dilation, groups=self.groups)

        return out



class BinaryConv1d_bw(nn.Conv1d):
    def __init__(self, in_channels: int, out_channels: int, kernel_size: _size_1_t, stride: _size_1_t = 1,
                 padding: Union[str, _size_1_t] = 0,
                 dilation: _size_1_t = 1,
                 groups: int = 1,
                 bias: bool = False,
                 padding_mode: str = 'zeros', ):
        super().__init__(in_channels, out_channels, kernel_size, stride=stride, padding=padding, dilation=dilation,
                         groups=groups, bias=bias, padding_mode=padding_mode)
        self.k = torch.tensor([10.]).float()
        self.t = torch.tensor([0.1]).float()
    def forward(self, input):
        a = input
        w = self.weight
        # -------IR-Net--------
        # ba = IR_Estimator().apply(a)
        # bw = IR_Estimator().apply(w)
        # -------STE----------
        # if self.training:
        # ba = BinarizeF().apply(a)
        # bw = BinarizeF().apply(w)
        # else:
        #     ba = torch.sign(a)
        #     bw = torch.sign(w)
        #-------IE-Net--------
        bw = OwnQuantize().apply(w, self.k.to(w.device), self.t.to(w.device))
        out = F.conv1d(input=a, weight=bw, bias=None, stride=self.stride,
                       padding=self.padding, dilation=self.dilation, groups=self.groups)

        return out
# ---------------------------------------IEE--------------------------------------
class OwnQuantize_a(Function):
    @staticmethod
    def forward(ctx, input, k, t):
        ctx.save_for_backward(input, k, t)
        out = torch.sign(input)
        return out

    @staticmethod
    def backward(ctx, grad_output):
        input, k, t = ctx.saved_tensors
        k = torch.tensor(1.).to(input.device)
        t = max(t, torch.tensor(1.).to(input.device))
        # grad_input = k * (1.4*t - torch.abs(t**2 * input))
        grad_input = k * (3 * torch.sqrt(t ** 2 / 3) - torch.abs(t ** 2 * input * 3) / 2)
        grad_input = grad_input.clamp(min=0) * grad_output.clone()
        return grad_input, None, None


class OwnQuantize(Function):
    @staticmethod
    def forward(ctx, input, k, t):
        ctx.save_for_backward(input, k, t)
        out = torch.sign(input)
        return out

    @staticmethod
    def backward(ctx, grad_output):
        input, k, t = ctx.saved_tensors
        # grad_input = k * (1.4*t - torch.abs(t**2 * input))
        grad_input = k * (3 * torch.sqrt(t ** 2 / 3) - torch.abs(t ** 2 * input * 3) / 2)
        grad_input = grad_input.clamp(min=0) * grad_output.clone()
        return grad_input, None, None

class BinaryConv1d_baw(nn.Conv1d):
    def __init__(self, in_channels: int, out_channels: int, kernel_size: _size_1_t, stride: _size_1_t = 1,
                 padding: Union[str, _size_1_t] = 0,
                 dilation: _size_1_t = 1,
                 groups: int = 1,
                 bias: bool = False,
                 padding_mode: str = 'zeros', ):
        super().__init__(in_channels, out_channels, kernel_size, stride=stride, padding=padding, dilation=dilation,
                         groups=groups, bias=bias, padding_mode=padding_mode)
        self.k = torch.tensor([10.]).float()
        self.t = torch.tensor([0.1]).float()

    def forward(self, input):
        a = input
        w = self.weight
        # -------IR-Net--------
        # ba = IR_Estimator().apply(a)
        # bw = IR_Estimator().apply(w)
        # -------STE----------
        # ba = BinarizeF().apply(a)
        # bw = BinarizeF().apply(w)
        # -------IEE----------
        bw = OwnQuantize().apply(w, self.k.to(w.device), self.t.to(w.device))
        ba = OwnQuantize_a().apply(a, self.k.to(w.device), self.t.to(w.device))
        out = F.conv1d(input=ba, weight=bw, bias=None, stride=self.stride,
                       padding=self.padding, dilation=self.dilation, groups=self.groups)

        return out

class BinaryLinear(nn.Linear):
    def __init__(self, in_features: int, out_features: int, bias: bool = False):
        super().__init__(in_features, out_features, bias)
        # self.binarize_a = BinaryActivation()
        # self.binarize_w = BinaryActivation()

    def forward(self, input):
        a = input
        w = self.weight
        ba = BinarizeF().apply(a)
        bw = BinarizeF().apply(w)
        out = F.linear(input=ba, weight=bw, bias=None)
        return out



class Bn_bin_conv_pool_block_Float(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, padding_value, pool_size,
                 pool_stride, ):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.bn = nn.BatchNorm1d(out_channels)  # default eps = 1e-5, momentum = 0.1, affine = True
        # 无bias
        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding,
                           bias=False,)
        self.relu = nn.ReLU()
        self.prelu = nn.PReLU()
        # self.htanh = nn.Hardtanh()
        # self.tanh = nn.Tanh()
        self.pool = nn.MaxPool1d(kernel_size=pool_size, stride=pool_stride)
        # self.binarize = BinaryActivation()
        # self.pad = nn.ConstantPad1d(padding=padding, value=padding_value)

    def forward(self, I):
        # I = self.pad(I)
        I = self.conv(I)
        I = self.pool(I)
        I = self.relu(I)
        I = self.bn(I)
        return I

class Bn_bin_conv_pool_block_bw(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, padding_value, pool_size,
                 pool_stride, ):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.bn = nn.BatchNorm1d(out_channels)  # default eps = 1e-5, momentum = 0.1, affine = True
        # 无bias
        self.conv = BinaryConv1d_bw(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=0,
                                    bias=False)
        self.relu = nn.ReLU()
        self.prelu = nn.PReLU()
        self.htanh = nn.Hardtanh()
        # self.tanh = nn.Tanh()
        self.pool = nn.MaxPool1d(kernel_size=pool_size, stride=pool_stride)
        # self.binarize = BinaryActivation()
        self.pad = nn.ConstantPad1d(padding=padding, value=padding_value)

    def forward(self, I):
        I = self.pad(I)
        I = self.conv(I)
        I = self.pool(I)
        I = self.prelu(I)
        I = self.bn(I)
        return I


class Bn_bin_conv_pool_block_baw(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, padding_value, pool_size,
                 pool_stride, ):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.bn = nn.BatchNorm1d(out_channels)  # default eps = 1e-5, momentum = 0.1, affine = True
        # 无bias
        self.conv = BinaryConv1d_baw(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=0,
                                    bias=False)
        self.relu = nn.ReLU()
        self.prelu = nn.PReLU()
        self.htanh = nn.Hardtanh()
        # self.tanh = nn.Tanh()
        self.pool = nn.MaxPool1d(kernel_size=pool_size, stride=pool_stride)
        self.pad = nn.ConstantPad1d(padding=padding, value=padding_value)

    def forward(self, I):
        I = self.pad(I)
        I = self.conv(I)
        I = self.pool(I)
        I = self.prelu(I)
        I = self.bn(I)
        return I

class BinActive(nn.Module):
    def __init__(self):
        super(BinActive, self).__init__()
        self.hardtanh = nn.Hardtanh(inplace=True)

    def forward(self, input):
        # output = self.hardtanh(input)
        # output = binarize(output)
        out_forward = torch.sign(input)
        out1 = self.hardtanh(input)
        output = out_forward.detach() - out1.detach() + out1
        return output




class WeightOperation:
    def __init__(self, model):
        self.model = model
        self.count_group_weights = 0
        self.weight = []
        self.saved_weight = []  # 保存weight的原始数据
        self.saved_alpha = []
        self.binarize = BinaryActivation()
        for m in model.modules():
            if isinstance(m, nn.Conv1d) or isinstance(m, nn.Linear):
                self.count_group_weights += 1
                self.weight.append(m.weight)
                self.saved_weight.append(m.weight.data)

    def WeightSave(self):
        for index in range(self.count_group_weights):
            self.saved_weight[index].copy_(self.weight[index].data)  # 把weight的值给了saved_weight,而后进行binary等操作

    def alpha_extract(self):
        for m in self.model.modules():
            if isinstance(m, nn.Conv1d) or isinstance(m, nn.Linear):
                self.saved_alpha.append(m.alpha)

    def WeightBinarize(self):
        for index in range(self.count_group_weights):
            self.weight[index].data = binarize(self.weight[index].data)  # 对应原model中的值也会跟着改变

    def WeightRestore(self):
        for index in range(self.count_group_weights):
            self.weight[index].data.copy_(self.saved_weight[index])

    def WeightGradient(self):
        for index in range(self.count_group_weights):
            self.weight[index].grad.data = self.weight[index].grad.data * self.saved_alpha[
                index].data.detach().transpose(0, 1)



class BinaryQuantize(Function):
    @staticmethod
    def forward(ctx, input):
        out = torch.sign(input)
        return out

    @staticmethod
    def backward(ctx, grad_output):
        grad_input = grad_output.clone()
        return grad_input


class BinaryQuantize_a(Function):
    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        out = torch.sign(input)
        return out

    @staticmethod
    def backward(ctx, grad_output):
        input = ctx.saved_tensors[0]
        grad_input = (2 - torch.abs(2 * input))
        grad_input = grad_input.clamp(min=0) * grad_output.clone()
        return grad_input

In [5]:
import torch
import os
import scipy.io as scio
from sklearn.model_selection import train_test_split
import numpy as np
from torch.utils.data import Dataset, DataLoader
from prettytable import PrettyTable


# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


def get_data(classes_num, device, test_size):
    labels = []
    X = list()
    y = list()
    if classes_num == 17:
        dataset_path = '/content/ECG_Dataset/ECG-17'
        labels = ['NSR', 'APB', 'AFL', 'AFIB', 'SVTA', 'WPW', 'PVC', 'Bigeminy',
                  'Trigeminy', 'VT', 'IVR', 'VFL', 'Fusion', 'LBBBB', 'RBBBB', 'SDHB', 'PR']

        for root, dirs, files in os.walk(dataset_path, topdown=False):
            for name in files:
                data_train = scio.loadmat(
                    os.path.join(root, name))  # 取出字典里的value

                # arr -> list
                data_arr = data_train.get('val')
                data_list = data_arr.tolist()
                X.append(data_list[0])  # [[……]] -> [ ]
                y.append(int(os.path.basename(root)[0:2]) - 1)

        X = torch.tensor(X, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.long)
        X_mean = torch.mean(X, dim=1, keepdim=True)
        X_std = torch.std(X, dim=1, keepdim=True)
        X = (X - X_mean) / X_std
        # X = X-X_mean
        X = X.reshape((1000, 1, 3600)).to(device)
        y = y.reshape((1000)).to(device)
    elif classes_num == 5:
        dataset_path = '/content/ECG_Dataset/ECG-5'
        labels = ['N', 'S', 'V', 'F', 'Q']

        for root, dirs, files in os.walk(dataset_path, topdown=False):
            for name in files:
                data_train = np.load(os.path.join(root, name), allow_pickle=True)  # 取出字典里的value

                data_list = data_train.tolist()
                X.append(data_list)  # [[……]] -> [ ]
                y.append(int(os.path.basename(root)[0:2]) - 1)


        X = torch.tensor(X, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.long)

        X_mean = torch.mean(X, dim=1, keepdim=True)
        X_std = torch.std(X, dim=1, keepdim=True)
        X = (X - X_mean) / X_std
        X = X.reshape((len(X), 1, 3600)).to(device)
        y = y.reshape((len(y))).to(device)


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

    return labels, X_train, X_test, y_train, y_test


class TrainDatasets(Dataset):
    def __init__(self, x_train, y_train):
        self.len = x_train.size(0)
        self.x_train = x_train
        self.y_train = y_train

    def __getitem__(self, index):
        return self.x_train[index], self.y_train[index]

    def __len__(self):
        return self.len


class TestDatasets(Dataset):
    def __init__(self, x_test, y_test):
        self.len = x_test.size(0)
        self.x_test = x_test
        self.y_test = y_test

    def __getitem__(self, index):
        return self.x_test[index], self.y_test[index]

    def __len__(self):
        return self.len


class Loader:
    def __init__(self, batch_size, classes_num, device, test_size):
        self.labels, self.x_train, self.x_test, self.y_train, self.y_test = get_data(classes_num, device, test_size)
        self.batch_size = batch_size
        self.train_dataset = TrainDatasets(self.x_train, self.y_train)
        self.test_dataset = TestDatasets(self.x_test, self.y_test)

    def loader(self):
        train_loader = DataLoader(dataset=self.train_dataset, batch_size=self.batch_size, shuffle=True)
        test_loader = DataLoader(dataset=self.test_dataset, batch_size=self.batch_size, shuffle=True)
        return self.labels, train_loader, test_loader

    def plot_train_test_splits(self):
        table = PrettyTable()
        table.field_names = ["", "ALL", "TRAIN", "TEST", "TEST RATIO"]
        ALL_SUM, TRAIN_SUM, TEST_SUM, TEST_RATIO = 0, 0, 0, 0
        for i in range(len(self.labels)):
            TRAIN = self.y_train.tolist().count(i)
            TEST = self.y_test.tolist().count(i)
            ALL = TRAIN + TEST
            TEST_RATIO = round(TEST / ALL, 3)
            table.add_row([self.labels[i], ALL, TRAIN, TEST, TEST_RATIO])

            ALL_SUM += ALL
            TRAIN_SUM += TRAIN
            TEST_SUM += TEST
        TEST_RATIO_SUM = round(TEST_SUM / ALL_SUM, 3)
        table.add_row(['Total', ALL_SUM, TRAIN_SUM, TEST_SUM, TEST_RATIO_SUM])
        return print(table)

In [6]:
"""
Contains functions for training and testing a PyTorch model.
"""
import numpy as np
import torch

from tqdm.auto import tqdm
from typing import Dict, List, Tuple
from torch.utils.tensorboard import SummaryWriter

# Create a writer with all default settings
writer = SummaryWriter()
from torchvision.ops.focal_loss import sigmoid_focal_loss

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device,
               weight_op) -> Tuple[float, float]:
    """Trains a PyTorch model for a single epoch.
    Turns a target PyTorch model to training mode and then
    runs through all of the required training steps (forward
    pass, loss calculation, optimizer step).
    Args:
    model: A PyTorch model to be trained.
    dataloader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    device: A target device to compute on (e.g. "cuda" or "cpu").
    Returns:
    A tuple of training loss and training accuracy metrics.
    In the form (train_loss, train_accuracy). For example:
    (0.1112, 0.8743)
    """
    # Put model in train mode
    model.train()

    # Setup train loss and train accuracy values
    train_loss, train_acc = 0, 0
    correct, total = 0, 0
    # Loop through data loader data batches
    for X, y in dataloader:
        # Send data to target device
        X, y = X.to(device), y.to(device)
        # weight_op.WeightSave()
        # weight_op.WeightBinarize()
        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate  and accumulate loss


        loss = loss_fn(y_pred, y)
        # loss = sigmoid_focal_loss(predicted.float(), y.float(), alpha=0.25, gamma=3, reduction='sum')

        train_loss += loss.item()
        # train_loss += loss
        # weight_op.WeightSave()
        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Calculate and accumulate accuracy metric across all batches
        # y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        # train_acc += (y_pred_class == y).sum().item() / len(y_pred)
        _, predicted = torch.max(y_pred.data, dim=1)
        total += len(y)
        correct += (predicted == y).sum().cpu().item()

    # Adjust metrics to get average loss and accuracy per batch
    train_loss = train_loss / len(dataloader)
    # train_acc = train_acc / len(dataloader)
    train_acc = correct / total
    return train_loss, train_acc


def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device,
              weight_op) -> Tuple[float, float]:
    """Tests a PyTorch model for a single epoch.
    Turns a target PyTorch model to "eval" mode and then performs
    a forward pass on a testing dataset.
    Args:
    model: A PyTorch model to be tested.
    dataloader: A DataLoader instance for the model to be tested on.
    loss_fn: A PyTorch loss function to calculate loss on the test data.
    device: A target device to compute on (e.g. "cuda" or "cpu").
    Returns:
    A tuple of testing loss and testing accuracy metrics.
    In the form (test_loss, test_accuracy). For example:
    (0.0223, 0.8985)
    """
    # Put model in eval mode
    model.eval()
    test_loss, test_acc = 0, 0
    correct, total = 0, 0
    # Turn on inference context manager
    with torch.inference_mode():
        # Loop through DataLoader batches
        for (X, y) in dataloader:
            # Send data to target device
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            test_pred_logits = model(X)
            # 2. Calculate and accumulate loss
            loss = loss_fn(test_pred_logits, y)
            # loss = sigmoid_focal_loss(predicted.float(), y.float(), alpha=0.25, gamma=3, reduction='mean')
            test_loss += loss.item()

            # Calculate and accumulate accuracy
            # test_pred_labels = test_pred_logits.argmax(dim=1)
            # test_acc += ((test_pred_labels == y).sum().item() / len(test_pred_labels))
            _, predicted = torch.max(test_pred_logits.data, dim=1)
            total += len(y)
            correct += (predicted == y).sum().cpu().item()

    # Adjust metrics to get average loss and accuracy per batch
    test_loss = test_loss / len(dataloader)
    # test_acc = test_acc / len(dataloader)
    test_acc = correct / total
    return test_loss, test_acc


def create_writer(experiment_name: str,
                  model_name: str,
                  extra: str = None) -> torch.utils.tensorboard.writer.SummaryWriter():
    """Creates a torch.utils.tensorboard.writer.SummaryWriter() instance saving to a specific log_dir.

    log_dir is a combination of runs/timestamp/experiment_name/model_name/extra.

    Where timestamp is the current date in YYYY-MM-DD format.

    Args:
        experiment_name (str): Name of experiment.
        model_name (str): Name of model.
        extra (str, optional): Anything extra to add to the directory. Defaults to None.

    Returns:
        torch.utils.tensorboard.writer.SummaryWriter(): Instance of a writer saving to log_dir.

    Example usage:
        # Create a writer saving to "runs/2022-06-04/data_10_percent/effnetb2/5_epochs/"
        writer = create_writer(experiment_name="data_10_percent",
                               model_name="effnetb2",
                               extra="5_epochs")
        # The above is the same as:
        writer = SummaryWriter(log_dir="runs/2022-06-04/data_10_percent/effnetb2/5_epochs/")
    """
    from datetime import datetime
    import os

    # Get timestamp of current date (all experiments on certain day live in same folder)
    timestamp = datetime.now().strftime("%Y-%m-%d")  # returns current date in YYYY-MM-DD format

    if extra:
        # Create log directory path
        log_dir = os.path.join("runs", timestamp, experiment_name, model_name, extra)
    else:
        log_dir = os.path.join("runs", timestamp, experiment_name, model_name)

    print(f"[INFO] Created SummaryWriter, saving to: {log_dir}...")
    return SummaryWriter(log_dir=log_dir)

def cpt_tk(epoch, epochs):
    "compute t&k in back-propagation"
    T_min, T_max = torch.tensor(1e-2).float(), torch.tensor(1e1).float()
    Tmin, Tmax = torch.log10(T_min), torch.log10(T_max)
    t = torch.tensor([torch.pow(torch.tensor(10.), Tmin + (Tmax - Tmin) / epochs * epoch)]).float()
    k = max(1 / t, torch.tensor(1.)).float()
    return t, k
import torch.nn as nn
def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device,
          writer: torch.utils.tensorboard.writer.SummaryWriter,
          weight_op,
          classes_num) -> Dict[str, List]:
    """Trains and tests a PyTorch model.
    Passes a target PyTorch models through train_step() and test_step()
    functions for a number of epochs, training and testing the model
    in the same epoch loop.
    Calculates, prints and stores evaluation metrics throughout.
    Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").
    Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for
    each epoch.
    In the form: {train_loss: [...],
              train_acc: [...],
              test_loss: [...],
              test_acc: [...]}
    For example if training for epochs=2:
             {train_loss: [2.0616, 1.0537],
              train_acc: [0.3945, 0.3945],
              test_loss: [1.2641, 1.5706],
              test_acc: [0.3400, 0.2973]}
    """
    # Create empty results dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
               }
    best_test_acc = 0.0
    best_train_acc = 0.0
    best_test_epoch = 0
    best_train_epoch = 0
    # Make sure model on target device
    model.to(device)
    model.to(device)
    conv_modules = []
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv1d):
            conv_modules.append(module)
    lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=1e-5,
                                                              last_epoch=-1)  # 0.000001
    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        t, k = cpt_tk(epoch=epoch, epochs=epochs)

        for module in conv_modules:
            module.k = k.cuda()
            module.t = t.cuda()
        train_loss, train_acc = train_step(model=model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer,
                                           device=device,
                                           weight_op=weight_op)

        test_loss, test_acc = test_step(model=model,
                                        dataloader=test_dataloader,
                                        loss_fn=loss_fn,
                                        device=device,
                                        weight_op=weight_op)
        # lr_scheduler.step()

        if test_acc > best_test_acc:
            best_test_acc = test_acc
            best_test_epoch = epoch + 1
            acc_str = "{:.2f}".format(best_test_acc * 100) + '%'
            if test_acc >= 0.95:
                # weight_op.WeightBinarize()
                save_filepath = f"ECG_Net_for_{classes_num}_{acc_str}.pth"
                save_model(model=model,
                           target_dir="models",
                           model_name=save_filepath)
        if train_acc > best_train_acc:
            best_train_acc = train_acc
            best_train_epoch = epoch + 1
            acc_str = "{:.2f}".format(best_test_acc * 100) + '%'

        # Print out what's happening
        print(
            f"Epoch: {epoch + 1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

        ### New: Experiment tracking ###
        # Add loss results to SummaryWriter
        if writer:
            # Add results to SummaryWriter
            writer.add_scalars(main_tag="Loss",
                               tag_scalar_dict={"train_loss": train_loss,
                                                "test_loss": test_loss},
                               global_step=epoch)
            writer.add_scalars(main_tag="Accuracy",
                               tag_scalar_dict={"train_acc": train_acc,
                                                "test_acc": test_acc},
                               global_step=epoch)
            # writer.add_graph(model=model,
            #                  # Pass in an example input
            #                  input_to_model=torch.randn(16, 1, 3600).to(device))

            # Close the writer
            writer.close()
        else:
            pass

    weight_op.WeightBinarize()
    print(model.state_dict())
    # Return the filled results at the end of the epochs

    print('best_test_acc: ', "{:.2f}".format(best_test_acc * 100) + '%', '\t epoch: ', best_test_epoch)
    print('best_train_acc: ', "{:.2f}".format(best_train_acc * 100) + '%', '\t epoch: ', best_train_epoch)
    return best_test_acc

In [7]:
import torch
import torch.nn as nn

class ECG_XNOR_Full_Bin(nn.Module):
    def __init__(self, block1, block2, block3, block4, block5, block6, block7, device):
        super(ECG_XNOR_Full_Bin, self).__init__()
        self.name = 'Full_Bin_ECG'
        self.device = device
        self.block1 = Bn_bin_conv_pool_block_baw(*block1)
        self.block2 = Bn_bin_conv_pool_block_baw(*block2)
        self.block3 = Bn_bin_conv_pool_block_baw(*block3)
        self.block4 = Bn_bin_conv_pool_block_baw(*block4)

        self.is_block5 = False
        self.is_block6 = False
        self.is_block7 = False
        if block5 is not None:
            self.block5 = Bn_bin_conv_pool_block_baw(*block5)
            self.is_block5 = True
        if block6 is not None:
            self.block6 = Bn_bin_conv_pool_block_baw(*block6)
            self.is_block6 = True
        if block7 is not None:
            self.block7 = Bn_bin_conv_pool_block_baw(*block7)
            self.is_block7 = True
        self.dropout0 = nn.Dropout(p=0.5)
    def forward(self, batch_data):
        batch_data = batch_data.clone().detach().requires_grad_(True).to(self.device)
        batch_data = self.block1(batch_data)
        batch_data = self.block2(batch_data)
        batch_data = self.block3(batch_data)
        batch_data = self.block4(batch_data)
        if self.is_block5:
            batch_data = self.block5(batch_data)
        if self.is_block6:
            batch_data = self.block6(batch_data)
        if self.is_block7:
            batch_data = self.block7(batch_data)
        batch_data = self.dropout0(batch_data)
        batch_data = batch_data.sum(dim=2)
        return batch_data

class ECG_XNOR_Ori(nn.Module):
    def __init__(self, block1, block2, block3, block4, block5, block6, block7, device):
        super(ECG_XNOR_Ori, self).__init__()
        self.name = 'Ori_Bin_ECG'
        self.device = device
        self.block1 = Bn_bin_conv_pool_block_bw(*block1)
        self.block2 = Bn_bin_conv_pool_block_baw(*block2)
        self.block3 = Bn_bin_conv_pool_block_baw(*block3)
        self.block4 = Bn_bin_conv_pool_block_baw(*block4)

        self.is_block5 = False
        self.is_block6 = False
        self.is_block7 = False
        if block5 is not None:
            self.block5 = Bn_bin_conv_pool_block_baw(*block5)
            self.is_block5 = True
        if block6 is not None:
            self.block6 = Bn_bin_conv_pool_block_baw(*block6)
            self.is_block6 = True
        if block7 is not None:
            self.block7 = Bn_bin_conv_pool_block_baw(*block7)
            self.is_block7 = True
        self.dropout0 = nn.Dropout(p=0.5)

    def forward(self, batch_data):
        batch_data = batch_data.clone().detach().requires_grad_(True).to(self.device)
        batch_data = self.block1(batch_data)
        batch_data = self.block2(batch_data)
        batch_data = self.block3(batch_data)
        batch_data = self.block4(batch_data)
        if self.is_block5:
            batch_data = self.block5(batch_data)
        if self.is_block6:
            batch_data = self.block6(batch_data)
        if self.is_block7:
            batch_data = self.block7(batch_data)
        batch_data = self.dropout0(batch_data)
        batch_data = batch_data.sum(dim=2)
        return batch_data



class ECG_Full_Float(nn.Module):
    def __init__(self, block1, block2, block3, block4, block5, block6, block7, device):
        super(ECG_Full_Float, self).__init__()
        self.name = 'Full_Bin_ECG'
        self.device = device
        self.block1 = Bn_bin_conv_pool_block_Float(*block1)
        self.block2 = Bn_bin_conv_pool_block_Float(*block2)
        self.block3 = Bn_bin_conv_pool_block_Float(*block3)
        self.block4 = Bn_bin_conv_pool_block_Float(*block4)

        self.is_block5 = False
        self.is_block6 = False
        self.is_block7 = False
        if block5 is not None:
            self.block5 = Bn_bin_conv_pool_block_Float(*block5)
            self.is_block5 = True
        if block6 is not None:
            self.block6 = Bn_bin_conv_pool_block_Float(*block6)
            self.is_block6 = True
        if block7 is not None:
            self.block7 = Bn_bin_conv_pool_block_Float(*block7)
            self.is_block7 = True
        self.dropout0 = nn.Dropout(p=0.5)
    def forward(self, batch_data):
        batch_data = batch_data.clone().detach().requires_grad_(True).to(self.device)
        batch_data = self.block1(batch_data)
        batch_data = self.block2(batch_data)
        batch_data = self.block3(batch_data)
        batch_data = self.block4(batch_data)
        if self.is_block5:
            batch_data = self.block5(batch_data)
        if self.is_block6:
            batch_data = self.block6(batch_data)
        if self.is_block7:
            batch_data = self.block7(batch_data)
        batch_data = self.dropout0(batch_data)
        batch_data = batch_data.mean(dim=2)
        return batch_data

In [8]:
import torch
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
    """Saves a PyTorch model to a target directory.
    Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
      either ".pth" or ".pt" as the file extension.
    Example usage:
    save_model(model=model_0,
               target_dir="models",
               model_name="05_going_modular_tingvgg_model.pth")
    """
    # Create target directory
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True,
                        exist_ok=True)

    # Create model save path
    assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
    model_save_path = target_dir_path / model_name

    # Save the model state_dict()
    print(f"[INFO] Saving model to: {model_save_path}")
    torch.save(obj=model,
             f=model_save_path)

In [9]:
!git lfs install
!git lfs pull

Git LFS initialized.
Not in a git repository.


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

import numpy as np
import random
import os

classes_num = 5
test_size = 0.2
if classes_num == 17:
    batch_size = 64
    lr = 0.002
    seed = 142
else:
    batch_size = 512
    lr = 0.02
    seed = 101


random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

loader = Loader(batch_size=batch_size, classes_num=classes_num, device=device, test_size=test_size)
labels, train_loader, test_loader = loader.loader()
# in_channels, out_channels,    kernel_size,     stride,    padding,   pad_value,   pool_size,  pool_stride
kernel_size, padding, poolsize =7, 5, 7
padding_value = 1
A = [[1,           8,           kernel_size,       2,       padding,       padding_value,       poolsize,        2],
     [8,          16,           kernel_size,       1,       padding,       padding_value,       poolsize,        2],
     [16,         32,           kernel_size,       1,       padding,       padding_value,       poolsize,        2],
     [32,         32,           kernel_size,       1,       padding,       padding_value,       poolsize,        2],
     [32,         64,           kernel_size,       1,       padding,       padding_value,       poolsize,        2],
     [64,         classes_num,  kernel_size,       1,       padding,       padding_value,       poolsize,        2],
     ]

model = ECG_XNOR_Ori(block1=A[0], block2=A[1], block3=A[2], block4=A[3],
                      block5=A[4] if len(A) > 4 else None,
                      block6=A[5] if len(A) > 5 else None,
                      block7=A[6] if len(A) > 6 else None,
                      device=device).to(device)
loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
print(device)
print(seed)

cuda:0
101


In [11]:
!pip install torchinfo

In [12]:
from torchinfo import summary
summary(model=model,
        input_size=(batch_size, 1, 3600),  # make sure this is "input_size", not "input_shape"
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

/tmp/ipython-input-2584412512.py:319: DeprecationWarning: <class '__main__.OwnQuantize'> should not be instantiated. Methods on autograd functions are all static, so you should invoke them on the class itself. Instantiating an autograd function will raise an error in a future version of PyTorch.
  bw = OwnQuantize().apply(w, self.k.to(w.device), self.t.to(w.device))
/tmp/ipython-input-2584412512.py:380: DeprecationWarning: <class '__main__.OwnQuantize'> should not be instantiated. Methods on autograd functions are all static, so you should invoke them on the class itself. Instantiating an autograd function will raise an error in a future version of PyTorch.
  bw = OwnQuantize().apply(w, self.k.to(w.device), self.t.to(w.device))
/tmp/ipython-input-2584412512.py:381: DeprecationWarning: <class '__main__.OwnQuantize_a'> should not be instantiated. Methods on autograd functions are all static, so you should invoke them on the class itself. Instantiating an autograd function will raise an e

Layer (type (var_name))                       Input Shape          Output Shape         Param #              Trainable
ECG_XNOR_Ori (ECG_XNOR_Ori)                   [512, 1, 3600]       [512, 5]             --                   True
├─Bn_bin_conv_pool_block_bw (block1)          [512, 1, 3600]       [512, 8, 898]        --                   True
│    └─ConstantPad1d (pad)                    [512, 1, 3600]       [512, 1, 3610]       --                   --
│    └─BinaryConv1d_bw (conv)                 [512, 1, 3610]       [512, 8, 1802]       56                   True
│    └─MaxPool1d (pool)                       [512, 8, 1802]       [512, 8, 898]        --                   --
│    └─PReLU (prelu)                          [512, 8, 898]        [512, 8, 898]        1                    True
│    └─BatchNorm1d (bn)                       [512, 8, 898]        [512, 8, 898]        16                   True
├─Bn_bin_conv_pool_block_baw (block2)         [512, 8, 898]        [512, 16, 448]      

In [13]:
weightOperation = WeightOperation(model)

In [14]:
num_epochs = 1000
best_test_acc = train(model=model,
      train_dataloader=train_loader,
      test_dataloader=test_loader,
      optimizer=optimizer,
      loss_fn=loss_fn,
      epochs=num_epochs,
      device=device,
      writer=False,
      weight_op=weightOperation,
      classes_num = classes_num)

print("-" * 50 + "\n")

  0%|          | 0/1000 [00:00<?, ?it/s]

/tmp/ipython-input-2584412512.py:319: DeprecationWarning: <class '__main__.OwnQuantize'> should not be instantiated. Methods on autograd functions are all static, so you should invoke them on the class itself. Instantiating an autograd function will raise an error in a future version of PyTorch.
  bw = OwnQuantize().apply(w, self.k.to(w.device), self.t.to(w.device))
/tmp/ipython-input-2584412512.py:380: DeprecationWarning: <class '__main__.OwnQuantize'> should not be instantiated. Methods on autograd functions are all static, so you should invoke them on the class itself. Instantiating an autograd function will raise an error in a future version of PyTorch.
  bw = OwnQuantize().apply(w, self.k.to(w.device), self.t.to(w.device))
/tmp/ipython-input-2584412512.py:381: DeprecationWarning: <class '__main__.OwnQuantize_a'> should not be instantiated. Methods on autograd functions are all static, so you should invoke them on the class itself. Instantiating an autograd function will raise an e

Epoch: 1 | train_loss: 6.0034 | train_acc: 0.4318 | test_loss: 2.9471 | test_acc: 0.7255
Epoch: 2 | train_loss: 2.2759 | train_acc: 0.6515 | test_loss: 1.1965 | test_acc: 0.7506
Epoch: 3 | train_loss: 1.7227 | train_acc: 0.6436 | test_loss: 1.3015 | test_acc: 0.7590
Epoch: 4 | train_loss: 1.2919 | train_acc: 0.6893 | test_loss: 0.8133 | test_acc: 0.7028
Epoch: 5 | train_loss: 0.9386 | train_acc: 0.7309 | test_loss: 0.7156 | test_acc: 0.8043
Epoch: 6 | train_loss: 0.7490 | train_acc: 0.7636 | test_loss: 0.4756 | test_acc: 0.8081
Epoch: 7 | train_loss: 0.6450 | train_acc: 0.7852 | test_loss: 0.5067 | test_acc: 0.8030
Epoch: 8 | train_loss: 0.6177 | train_acc: 0.7794 | test_loss: 0.4992 | test_acc: 0.8191
Epoch: 9 | train_loss: 0.5406 | train_acc: 0.8151 | test_loss: 0.5560 | test_acc: 0.7468
Epoch: 10 | train_loss: 0.5207 | train_acc: 0.8203 | test_loss: 0.6649 | test_acc: 0.8114
Epoch: 11 | train_loss: 0.4710 | train_acc: 0.8361 | test_loss: 0.4779 | test_acc: 0.8572
Epoch: 12 | train_l